make a gene table with:
contig, contig_length, gene_name, gene_start, gene_stop, gene_orient, Evidence_source, evidence_acession, evidence_description, virus_score(or category)

In [1]:
import os
import re
import sys
import pandas as pd
import glob
import numpy as np

import itertools
from itertools import tee
import csv

import statistics
from statistics import mean
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import math
import collections

In [2]:
repeat_table = "/Users/u241374/mike_tisza/sandbox/ct2_test_731c/ct2_tmp/hallmark_contigs_terminal_repeat_summary.tsv"

phan_tab_directory = "/Users/u241374/mike_tisza/sandbox/ct2_test_731c/ct2_tmp/reORF/phan_split"

prod_tab_directory = "/Users/u241374/mike_tisza/sandbox/ct2_test_731c/ct2_tmp/reORF/prod_split"

first_pyhmmer_table = "/Users/u241374/mike_tisza/sandbox/ct2_test_731c/ct2_tmp/reORF_pyhmmer/pyhmmer_report_AAs.tsv"

second_pyhmmer_table = "/Users/u241374/mike_tisza/sandbox/ct2_test_731c/ct2_tmp/second_reORF_pyhmmer/pyhmmer_report_AAs.tsv"

mmseqs_CDD_table = "/Users/u241374/mike_tisza/sandbox/ct2_test_731c/ct2_tmp/third_reORF_combined/summary_no2_AAs_vs_CDD.besthit.tsv"

viral_cdds_list = "/Users/u241374/mike_tisza/cmmr_repos/Cenote-Taker2/viral_cdds_and_pfams_191028.txt"

In [3]:
#just_gene_df = pd.DataFrame(columns=["gene_name", "gene_start", "gene_stop", "gene_orient"])
try:

    phan_files = glob.glob(os.path.join(phan_tab_directory, "*.bed"))

    df_from_each_phan = (pd.read_csv(phan, sep = "\t", header = None,
                                    names = ["contig", "gene_start", "gene_stop", "gene_name", 
                                            "gene_score", "gene_orient"])
                                    for phan in phan_files)
    phan_gene_df = pd.concat(df_from_each_phan, ignore_index=True)

    phan_gene_df = phan_gene_df.drop("gene_score", axis = 1)

except:
    print("no phanotate tables")



In [4]:
try:

    prod_files = glob.glob(os.path.join(prod_tab_directory, "*.gff"))

    df_from_each_prod = (pd.read_csv(prod, sep = "\t", header = None, comment='#',
                                    names = ["contig", "gene_caller", "feature_type", "gene_start", 
                                            "gene_stop", "score", "gene_orient", "frame", "attribute"])
                                    for prod in prod_files)
    prod_gene_df = pd.concat(df_from_each_prod, ignore_index=True)

    prod_gene_df = prod_gene_df[["contig", "gene_start", "gene_stop", "attribute", "gene_orient"]]
except:
    print("no prodigal tables")
    prod_gene_df = pd.DataFrame()

no prodigal tables


In [5]:
if not phan_gene_df.empty and not prod_gene_df.empty:
    print("both")
    both_list = [phan_gene_df, prod_gene_df]
    just_gene_df = pd.concat(both_list, ignore_index=True)
elif not phan_gene_df.empty:
    print("phan")
    just_gene_df = phan_gene_df
elif not prod_gene_df.empty:
    print("prod")
    just_gene_df = prod_gene_df


phan


In [6]:
try:
    length_df = pd.read_csv(repeat_table, sep = "\t")[['contig', 'out_length_contig']]

    length_df = length_df.rename(columns={"out_length_contig": "contig_length"})
except:
    print("nope")

In [7]:
try:
    basal_df = just_gene_df.merge(length_df, on = "contig", how = "left")
except:
    print("nope")

In [8]:
first_pyh_df = pd.read_csv(first_pyhmmer_table, sep = "\t")[['ORFquery', 'target']]

first_pyh_df.head()

,ORFquery,target
0,ct2_test_731c43597_90(-),baits_missed.111/PDB:3KDR_A-HK97-Family-Phage-...
1,ct2_test_731c4767_65(+),tmp1_clust12/Uniprot:R5NKC2_9FIRM-tail-tape-me...
2,ct2_test_731c36689_13(-),podo_cluster.10/PFAM:PF11134.7--Phage-stabilis...
3,ct2_test_731c2265_7(-),seeker_phage_cluster19/PDB:3CPE_A-Terminase
4,ct2_test_731c37702_35(+),cdd_cluster.55/CDD:PHA02535-terminase-ATPase-s...


In [9]:
# gene_name, Evidence_source, evidence_acession, evidence_description, virus_score(or category)

try:
    first_pyh_df = pd.read_csv(first_pyhmmer_table, sep = "\t")[['ORFquery', 'target']]


    first_pyh_df["lpar_pos"] = first_pyh_df["ORFquery"].str.find("(")

    first_pyh_df["gene_name"] = first_pyh_df.apply(lambda x: x["ORFquery"][0:x["lpar_pos"]], axis = 1)

    first_pyh_df["slash_pos"] = first_pyh_df["target"].str.find("/")
    first_pyh_df["fdash_pos"] = first_pyh_df["target"].str.find("-")


    first_pyh_df["evidence_acession"] = first_pyh_df.apply(
        lambda x: x["target"][x["slash_pos"]+1:x["fdash_pos"]], axis = 1)

    first_pyh_df["evidence_description"] = first_pyh_df.apply(lambda x: x["target"][x["fdash_pos"]+1:], axis = 1)

    first_pyh_df = first_pyh_df[['gene_name', 'evidence_acession', 'evidence_description']]

    first_pyh_df['Evidence_source'] = 'hallmark_hmm'

    first_pyh_df['vscore_category'] = 'common_virus'

except:
    print("nope")



In [10]:
try:
    second_pyh_df = pd.read_csv(second_pyhmmer_table, sep = "\t")[['ORFquery', 'target']]


    second_pyh_df["lpar_pos"] = second_pyh_df["ORFquery"].str.find("(")

    second_pyh_df["gene_name"] = second_pyh_df.apply(lambda x: x["ORFquery"][0:x["lpar_pos"]], axis = 1)

    second_pyh_df["slash_pos"] = second_pyh_df["target"].str.find("/")
    second_pyh_df["fdash_pos"] = second_pyh_df["target"].str.find("-")


    second_pyh_df["evidence_acession"] = second_pyh_df.apply(
        lambda x: x["target"][x["slash_pos"]+1:x["fdash_pos"]], axis = 1)

    second_pyh_df["evidence_description"] = second_pyh_df.apply(lambda x: x["target"][x["fdash_pos"]+1:], axis = 1)

    second_pyh_df = second_pyh_df[['gene_name', 'evidence_acession', 'evidence_description']]

    second_pyh_df['Evidence_source'] = 'common_virus_hmm'

    second_pyh_df['vscore_category'] = 'common_virus'

except:
    print("nope")

In [11]:
cdd_df = pd.read_csv(mmseqs_CDD_table, sep = "\t")[['query', 'target', 'description']]

cdd_df["lpar_pos"] = cdd_df["query"].str.find("(")

cdd_df["gene_name"] = cdd_df.apply(lambda x: x["query"][0:x["lpar_pos"]], axis = 1)

cdd_df = cdd_df[['gene_name', 'target', 'description']]

cdd_df = cdd_df.rename(columns={"target": "evidence_acession", "description" : "evidence_description"})

cdd_df['Evidence_source'] = 'mmseqs_cdd'

#cdd_df['vscore_category'] = 'nonviral_gene'

#cdd_df['vscore_category'] = np.where(cdd_df['target']
#                               .str.contains(
#                                   "PHA0",
#                                   case = False), 'common_virus', cdd_df['vscore_category'])




cdd_df.head()

,gene_name,evidence_acession,evidence_description,Evidence_source
0,ct2_test_731c2983_13,pfam14902,Domain of unknown function (DUF4494). This fam...,mmseqs_cdd
1,ct2_test_731c3157_42,pfam08877,MepB protein. MepB is a functionally uncharact...,mmseqs_cdd
2,ct2_test_731c3157_43,cd01031,ClC chloride channel EriC. This domain is fou...,mmseqs_cdd
3,ct2_test_731c3157_44,PRK00015,ribonuclease HII; Validated,mmseqs_cdd
4,ct2_test_731c3157_45,TIGR03596,ribosome biogenesis GTP-binding protein YlqF. ...,mmseqs_cdd


In [12]:
virlist_df = pd.read_csv(viral_cdds_list, header = None, names = ["evidence_acession"])

virlist_df['vscore_category'] = 'common_virus'

In [13]:
comb_cdd_df = cdd_df.merge(virlist_df, on = "evidence_acession", how = "left")

comb_cdd_df['vscore_category'] = np.where(comb_cdd_df['evidence_acession']
                               .str.contains(
                                   "PHA0",
                                   case = False), 'common_virus', comb_cdd_df['vscore_category'])

comb_cdd_df['vscore_category'] = np.where(comb_cdd_df['vscore_category'].isna(), 'nonviral_gene',
                                          comb_cdd_df['vscore_category'])

In [14]:
gene_ann_list = []

if not first_pyh_df.empty:
    gene_ann_list.append(first_pyh_df)

if not second_pyh_df.empty:
    gene_ann_list.append(second_pyh_df)

if not comb_cdd_df.empty:
    gene_ann_list.append(comb_cdd_df)

try:
    gene_ann_df = pd.concat(gene_ann_list, ignore_index=True)

    contig_gene_df = basal_df.merge(gene_ann_df, on = "gene_name", how = "left")

    contig_gene_df['vscore_category'] = np.where(contig_gene_df['vscore_category'].isna(), 'hypothetical_protein',
                                          contig_gene_df['vscore_category'])
    
    contig_gene_df['evidence_description'] = np.where(contig_gene_df['evidence_description'].isna(), 'hypothetical protein',
                                          contig_gene_df['evidence_description'])

except:
    print("nope")

In [34]:
####define scoring scheme and important variables
id_list = ['common_virus','hypothetical_protein','nonviral_gene','intergenic']
score_list = [10,5,-3,0]

keys = id_list
values = score_list
domain_dictionary = dict(zip(keys,values))

threshold = 0
window = 5000
sliiiiiide_to_the_right = 50


grouped_df = contig_gene_df.query("contig_length >= 10000").groupby('contig')

for name, group in grouped_df:

    count=0
    count_start=-sliiiiiide_to_the_right

    contig_length1 = group['contig_length'].agg(pd.Series.mode)


    vscore_list = list([0] * int(contig_length1.iloc[0]))

    total_len = int(len(vscore_list))

    for index, row in group.iterrows():
        #print(row['vscore_category'], domain_dictionary[row['vscore_category']])
        S = domain_dictionary[row['vscore_category']]
        vscore_list[row['gene_start']:row['gene_stop']] = [S] * (row['gene_stop'] - row['gene_start'])

    #letter_list = [domain_dictionary[k] for k in x] #convert to scores
    seq_score_nope = sum(vscore_list)
    #avg_score = mean(letter_list)
    blocks = int(((len(vscore_list) - window) / sliiiiiide_to_the_right) + 1)
    blocks_2 = blocks + 2 #need this otherwise the last (incomplete/little) block will be cut off!
    #print("you will have " + str(blocks_2) + " windows")

    #cols = ['Window', 'Position start', 'Position stop','Pass/Fail', 'Score', 'V_count', 'X_count', 'Z_count', 'Y_count']
    #dat = pd.DataFrame(columns = cols)
    dat_list = []
    for i in range(0, blocks_2 * sliiiiiide_to_the_right, sliiiiiide_to_the_right):
        score_result = sum(vscore_list[i:i+window])
        new_let_list = vscore_list[i:i+window]

        if score_result >= 0 :
            PF_result = "pass"
        else:
            PF_result = "fail"

        #counts for later
        V_count = new_let_list.count(10)
        X_count = new_let_list.count(5)
        Y_count = new_let_list.count(-3)
        Z_count = new_let_list.count(0)

        #vars for count columns
        count = count +1
        count_start += sliiiiiide_to_the_right #same as c_s = c_s + siiii...
        count_stop = count_start+window

        #dat.index.name = 'Window'
        #let's plot things!
        dat_list.append([count, count_start, count_stop, PF_result, score_result, V_count,
                          X_count, Y_count, Z_count])
        #dat.index.name = 'Window'

    #outname = (str(file.name)+".tableout.tsv")

    #FIGURES
    pdf_outname = os.path.join("/Users/u241374/mike_tisza/sandbox/figures_out", name+".figures.pdf")
    #Character ocunts plot
    #figures = PdfPages(pdf_outname)
    df_0 = pd.DataFrame(dat_list, columns=['Window', 'Position start', 'Position stop',
                                             'Pass/Fail', 'Score', 'V_count', 'X_count', 
                                             'Z_count', 'Y_count'])
    #dat.to_csv(outname, sep='\t', index=False)
    #MAIN DATAFRAME CREATED, STORED IN df_0

    #Now let's make the smoothed plot

    #median_0 = df_0['Annotation'].median()
    x = df_0['Window']
    y = np.array(df_0['Score'])
    l = df_0['Window'].count()
    df_empty = pd.DataFrame(index=range(l),columns=range(1))
    for col in df_empty.columns:
        df_empty[col].values[:] = 0

    zero=df_empty[0]

    def smooth(y, box_pts):
        box = np.ones(box_pts)/box_pts
        y_smooth = np.convolve(y, box, mode='same')
        return y_smooth
    #smooth_val == box_plts
    smooth_val = 100 #####we can change this if we want!


    #statement for handling short sequences (error called if len(y) < smoothing value)
    if len(y) <= smooth_val:
        smooth_val = int(0.5 * len(y))
    else:
        smooth_val = int(smooth_val)

    smoth = smooth(y,smooth_val)
    idx = np.argwhere(np.diff(np.sign(zero - smoth))).flatten()
    df = pd.DataFrame(zero[idx])
    df = df.reset_index()
    #we will save to figures, but first we need to do the validation steps

    #This is for validating if region is + or -
    df.loc[-1] = 1  # adding a row for first position
    df.index = df.index + 1  # shifting index
    df = df.sort_index()

    #df.iloc[-1] = len(y)

    #last position as last row
    #print(df['index'])
    df.sort_values(by=['index']) #need to sort first otherwise +1 belwo will break things
    new_list = pd.DataFrame(df['index'] + 1) #df['index'][:-1] + 1 #add +1 to all for next position is +/-, except for last position, will throw erre - so it deletes it, we'll add it in later
    #print(new_list)

    #the_val_to_add = df.iloc[-1] - 1

    #new_list = new_list.append(df.iloc[-1] - 1) #beacuse of +1 transformation few lines above
    new_list_2 = new_list['index']

    #new_list = new_list.append(last_val_to_append, ignore_index=True)

    new_y_val = list(smoth[new_list])   #find position y on smooth line


    #assigning pos / neg for that +1 position
    pos_neg_results = []
    for i in new_y_val:
        if i > 0:
            result = '+'
        else:
            result = '-'
        pos_neg_results.append(result)

    #pos_neg_results.append('N/A') #the last value needs this - not anymore

    #print(pos_neg_results)
    #creating dataframe for next steps
    df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True) #to delete last column, unnamed so tricky to get rid of (?) this does it tho
    df['+/- to the right'] = pos_neg_results
    #print(df['+/- to the right'])
    #append +/- and start stop coords from original table
    df.rename(columns={'index': 'Window'}, inplace=True)

    df['Window']=df['Window'].astype(int)
    df_0['Window']=df_0['Window'].astype(int)

    merged_df = df.merge(df_0, how = 'inner', on = ['Window'])

    merged_df = merged_df.drop(['Pass/Fail','Score','V_count','X_count','Z_count','Y_count'], axis = 1)
    merged_df['Chunk_end'] = 'none'
    merged_df['Window midpoint'] = merged_df.iloc[:,[2,3]].median(axis=1)
    merged_df['Window midpoint'] = merged_df['Window midpoint'].astype(int)

    #df edits to accomodate this:
    #we are duplicating the last row of the df to handle a trailing + chunk (w/ no y=0 intercept to close the chunk)
    #merged_df = merged_df.append(merged_df[-1:])
    merged_df = pd.concat([merged_df, merged_df[-1:]])
    #now need to make it read actual last stop position (this os not rounded per window like the other coords)
    merged_df = merged_df.replace(merged_df.iloc[-1][3],(total_len+1))
    print(merged_df)

    #now let's get the coordinates for the > 0 'chunks'

    #iterate over for true hit testing
    def pairwise(iterable):
        "s -> (s0,s1), (s1,s2), (s2, s3), ..."
        a, b = tee(iterable)
        next(b, None)
        return zip(a, b)



    #this is to define the chunks, accounting for all the ways the graph can look
    #note: leading and trailing here mean a chunk at the start or end of the graph that
    ddf_list = []

    for (i1, row1), (i2, row2) in pairwise(merged_df.iterrows()):
        #for a leading chunk
        if row1['+/- to the right'] == '+' and \
            row1["Position start"] == 0 and \
            row1["Position stop"] != (total_len + 1):
                ddf = ["Chunk_" + str(i1), row1["Position start"], row2["Window midpoint"]]
                ddf_list.append(ddf)
        #for a contained chunk
        if row1['+/- to the right'] == '+' and \
            row1["Position start"] != 0 and \
            row1["Position stop"] != (total_len + 1):
                ddf = ["Chunk_" + str(i1), row1["Window midpoint"], row2["Window midpoint"]]
                ddf_list.append(ddf)
        #3. for a trailing chunk
        if row1['+/- to the right'] == '+' and \
            row1["Position start"] != 0 and \
            row1["Position stop"] == (total_len + 1): #old = merged_df.iloc[0,3]
                ddf = ["Chunk_" + str(i1), row1["Window midpoint"], row2["Position stop"]]
                ddf_list.append(ddf)

        #4. for graphs with no leading and no trailing chunk (for graphs with no y = 0 intercept -> this is is
        #a differently-defined statemnt below b/c the empty file gets appended w/ stuff above from older files when
        #it's in the loop, ALSO the criterion gets fulfilled by contained cunks which means duplicate csv rows for chunks (defined diffrently to specifiy the rules)
        if merged_df.iloc[0,1] == '+' and \
            merged_df.iloc[0,2] == 0 and \
            merged_df.iloc[0,3] == (total_len + 1): #if first column last(2nd row) == last -1 then its one chunk
                rep_list = [('Chunk_0', '0', (total_len+1))]
                ddf_list = rep_list
        else:
                ddf_list = ddf_list

    #print(merged_df)
    #print(ddf_list)

    #make chunk csv
    chunk_df = pd.DataFrame(ddf_list)
    #this_name = str(file.name+"_chunk_coordinates.csv") #used to be fna_name

    chunk_sum_file = os.path.join("/Users/u241374/mike_tisza/sandbox/figures_out", name+".chunks.tsv")

    chunk_df.to_csv(chunk_sum_file, sep = "\t", index = False)
    ###Find optimal location on plot to place validation marker
    #read in virus table

    #file_name_just_stem = file.name[:-4]

    #vir_bait_table = str(actual_file_name_temp+'.VIRUS_BAIT_TABLE.txt')
    #with open(vir_bait_table, 'r') as csvfile:
    #    reader = csv.reader(csvfile, delimiter='\t')
    #    lines = list(reader)

    #vir_bait_table = pd.DataFrame(lines)
    #vir_bait_table['median'] = round(vir_bait_table[[1,2]].median(axis=1))
    #vir_bait_table_med_list = list(vir_bait_table['median'])
    #print(vir_bait_table_med_list)


    vir_bait_table = group[['gene_start', 'gene_stop', 'Evidence_source']]\
        .query("Evidence_source == 'hallmark_hmm'")
    vir_bait_table['mean'] = round(vir_bait_table[['gene_start', 'gene_stop']].mean(axis=1))
    vir_bait_table_med_list = list(vir_bait_table['mean'])
    #vir_bait_table_med_list = list(round(vir_bait_table[[1,2]].median(axis=1)))

    points_list = []
    for item in vir_bait_table_med_list:
        eq = round(((item - 2500) + 50) / 50)
        if eq >= len(x):
            plot_point = (len(x) - 1) #1 because it can't = len, has to be less
        else:
            plot_point = eq
        #plot_point = round(((item - 2500) + 50) / 50) #this must stay at = window length (not half like we had talked about, it makes illogical values...basically if the coordinate is towards the end, applying a window 'inbetween' can be out of bounds)
        points_list.append(plot_point)

    new_points_list = [1 if i <=0 else i for i in points_list]

    #print(points_list) #each item represents/is the best/closet window that captures the viral hallmark region
    df_0['smoothy'] = smooth(df_0['Score'],100)
    zero=df_empty[0]

    figures = PdfPages(pdf_outname)

    x2 = (points_list)
    plt.plot(x, y, 'o', ms=0.6)
    plt.axhline(0, 0, l)
    #plt.plot(x, smooth(y,3), 'r-', lw=2)
    #p = smooth(y,100)
    plt.plot(x, df_0['smoothy'], 'c', lw=2)
    plt.plot(x, df_0['smoothy'], 'y', markevery = (new_points_list), ms=11.0, marker = '*')
    plt.title("Viral region calls")
    plt.xlabel('Window')
    plt.ylabel('Score')
    plt.rc('axes', titlesize=6.8)     # fontsize of the axes title
    plt.rc('xtick', labelsize=5)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=5)    # fontsize of the tick labels
    plt.rc('legend', fontsize=5)    # legend fontsize
    plt.rc('figure', titlesize=8)  # fontsize of the figure title
    plt.grid(True)
    idx = np.argwhere(np.diff(np.sign(zero - smooth(y,100)))).flatten()
    plt.plot(x[idx], zero[idx],  'ro', ms=5.0)
    #plt.plot(x[idx], zero[idx], markevery= (points_list), ms=9.0, marker = 'X', color = 'y')
    #plt.plot()
    df = pd.DataFrame(zero[idx])
    plt.plot()
    plt.savefig(figures, format='pdf')
    plt.close()
    df = df.reset_index()
    #print(df)

    mycol = (["#e7ba52", "#637939", "#7b4173", "#d6616b"])
    df_0[['V_count','X_count','Y_count','Z_count']].plot(color = mycol) #same = dat.plot(y=['X_count','N_count','R_count','V_count']) , plt.show()        plt.grid(True)
    plt.grid(True)
    plt.xlabel('Window')
    plt.ylabel('Count')
    plt.title('Character counts')
    plt.rc('axes', titlesize=6.8)     # fontsize of the axes title
    plt.rc('xtick', labelsize=5)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=5)    # fontsize of the tick labels
    plt.rc('legend', fontsize=5)    # legend fontsize
    plt.rc('figure', titlesize=8)  # fontsize of the figure title
    plt.savefig(figures, format='pdf')
    plt.close()

    figures.close()

    


   Window +/- to the right  Position start  Position stop Chunk_end  \
0       1                +               0          12963      none   
0       1                +               0          12963      none   

   Window midpoint  
0             2500  
0             2500  
   Window +/- to the right  Position start  Position stop Chunk_end  \
0       1                +               0          17547      none   
0       1                +               0          17547      none   

   Window midpoint  
0             2500  
0             2500  
   Window +/- to the right  Position start  Position stop Chunk_end  \
0       1                +               0          62982      none   
0       1                +               0          62982      none   

   Window midpoint  
0             2500  
0             2500  
   Window +/- to the right  Position start  Position stop Chunk_end  \
0       1                +               0          19539      none   
0       1                + 